# benjihillard dictionary processing

I found a french dictionary here: https://github.com/Kartmaan/french-language-tools?tab=readme-ov-file

In [1]:
import pandas as pd
from tqdm import tqdm
import ast

In [2]:
raw = pd.read_csv("../../../RawData/Dictionaries/French/kartmann.csv")

In [3]:
raw.sample(4)

,Mot,Définitions
719803,Jarselaine,"['Habitante de Jarsy, commune française située..."
392350,Démenotterions,['Première personne du pluriel du conditionnel...
367606,Enamouriez,"[""Deuxième personne du pluriel de l'indicatif ..."
613539,Bethenyate,['Féminin singulier de bethenyat.']


## Multiple definitions

All the definitions of a word are collected in the one entry, we'll have to chop them up.


In [4]:
raw[raw.Mot == "Livre"]["Définitions"].tolist()[0]

'[\'Ensemble de pages reliées entre elles et dotées de signes destinés à être lus.\', "Ouvrage de l\'esprit, en prose ou en vers, d\'assez grande étendue pour faire au moins un volume.", \'Écrit sur une face et se présentant sous forme de rouleau.\', \'Ensemble de feuilles de parchemin ou de papier écrites des deux côtés et rassemblées en cahiers, codex.\', \'Ouvrage imprimé, relié ou broché, non périodique, comportant un assez grand nombre de pages.\', "Ce type d\'ouvrage, publié sur un autre support.", \'Sous-partie, division de ces ouvrages.\', \'Terme générique faisant référence à un ouvrage servant de base à une religion et considéré comme sacré.\', "Commerce Registre sur lequel on inscrit ce qu\'on reçoit et ce qu\'on dépense, ce qu\'on achète et ce qu\'on vend, ses dettes actives et ses dettes passives, etc.", "Diplomatie Accompagné d\'une épithète de couleur (désignant la couverture) : Recueil de pièces diplomatiques, publié par le gouvernement d\'un pays.", "Secteur d\'activit

In [5]:
mot = "Livre"
D = raw.loc[raw['Mot'] == mot]['Définitions']
D = eval(D.values[0])

print("Mot : ", mot)
for idx, definition in enumerate(D):
    print(f"{idx+1}) {definition}")

Mot :  Livre
1) Ensemble de pages reliées entre elles et dotées de signes destinés à être lus.
2) Ouvrage de l'esprit, en prose ou en vers, d'assez grande étendue pour faire au moins un volume.
3) Écrit sur une face et se présentant sous forme de rouleau.
4) Ensemble de feuilles de parchemin ou de papier écrites des deux côtés et rassemblées en cahiers, codex.
5) Ouvrage imprimé, relié ou broché, non périodique, comportant un assez grand nombre de pages.
6) Ce type d'ouvrage, publié sur un autre support.
7) Sous-partie, division de ces ouvrages.
8) Terme générique faisant référence à un ouvrage servant de base à une religion et considéré comme sacré.
9) Commerce Registre sur lequel on inscrit ce qu'on reçoit et ce qu'on dépense, ce qu'on achète et ce qu'on vend, ses dettes actives et ses dettes passives, etc.
10) Diplomatie Accompagné d'une épithète de couleur (désignant la couverture) : Recueil de pièces diplomatiques, publié par le gouvernement d'un pays.
11) Secteur d'activité regro

## Capitalisation

Already we see a problem - all words are capitalised, however some words' meaning changes based on capitalisation, such as the constellation vs disease cancer.

We assume the root of a word to be the uncapitalised form, unless it is a proper noun.

## Conclusion
 
We'll lowercase everything and separate the defintions, we accept that dictionaries can be fallable. 

In [6]:
def lowercase_first_letter(s):
    return s[0].lower() + s[1:] if s else s

In [7]:
raw['Mot'] = raw['Mot'].apply(lowercase_first_letter) # lowercase
raw.reset_index(inplace = True)

In [8]:
raw.sample(3)

,index,Mot,Définitions
220419,220419,étirasses,"[""Deuxième personne du singulier de l'imparfai..."
540002,540002,charognasse,"[""Première personne du singulier de l'imparfai..."
121706,121706,pégriots,['Masculin pluriel de pégriot.']


In [9]:
tqdm.pandas(desc="Converting Definitions")
raw['Définitions'] = raw['Définitions'].progress_apply(ast.literal_eval) # safer than eval

# Explode the 'Définitions' list so each definition gets its own row

expanded_raw = raw.explode('Définitions')

C:\Users\Matthew\anaconda3\lib\site-packages\tqdm\std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
Converting Definitions: 100%|███████████████████████████████████████████████| 973671/973671 [00:12<00:00, 77400.17it/s]


In [10]:
expanded_raw = expanded_raw.sort_values('Mot').reset_index(drop = True)

In [11]:
expanded_raw.drop(["index"], axis = 1, inplace = True)

In [12]:
expanded_raw.reset_index(inplace = True)

In [13]:
expanded_raw.head()

,index,Mot,Définitions
0,0,&amp;,variante ortho de et.
1,1,&amp;c.,variante ortho de etc.
2,2,'alif,variante ortho de alif.
3,3,'tain,Forme familière de putain.
4,4,'tis,Masculin pluriel de 'ti.


In [14]:
processed = expanded_raw.rename(columns = {"index" : "ID", "Mot" : "Word", "Définitions" : "Definition"})

In [15]:
def remove_newline(s):
    return s.replace('\n', '')

processed["Word"] = processed["Word"].apply(remove_newline)
processed["Definition"] = processed["Definition"].apply(remove_newline)

In [16]:
save_dir = "../../../ProcessedData/Dictionaries/French/"
import os
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

In [17]:
processed.to_csv(save_dir + "kartmann.csv", encoding='utf-16', index=False)